# Asignación Financiera

## Librerias necesarias

In [181]:
import pandas as pd
import os
import pickle
import json
import xlsxwriter
from xlsxwriter.utility import xl_range, xl_rowcol_to_cell
import re
from typing import Dict, List, Any
import itertools
from firebase_admin import credentials, firestore
import firebase_admin
import excel2img
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

## Datos del proyecto en análisis

In [182]:
proyecto="misanayoc"
mes=5
anio=2025

## Datos de Firestore

In [183]:
# Ruta a tu archivo de clave privada descargado
cred = credentials.Certificate("../credentials_firestore.json")

# Inicializa sólo si no hay ninguna app por defecto
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred)

# Obtiene una instancia de la base de datos Firestore
db = firestore.client()

In [184]:
data_mantenimiento=db.collection("rutinarios").document(proyecto).get()
if data_mantenimiento.exists:
    print(data_mantenimiento.to_dict())
else:
    print("No existe el documento.")

{'otros': {'tipo_superficie': 'Asfaltado', 'estado_conservacion_via': 'regular'}, 'other': {'tipo_superficie': 'Asfaltado', 'estado_conservacion_via': 'regular'}, 'expediente': {'codigo_ruta': 'AR-683', 'monto_total': 206413.0, 'coordenadas': {'fin': {'hemisferio': 'S', 'y': 8270075.15, 'progresiva': 32252.0, 'zona_letra': None, 'x': 220383.97, 'datum': 'WGS84', 'altitud': 3629, 'zona': None}, 'inicio': {'hemisferio': 'S', 'y': 8281146.79, 'progresiva': 0, 'zona_letra': None, 'x': 226316.6, 'datum': 'WGS84', 'altitud': 3820, 'zona': None}}, 'nombre': 'MANTENIMIENTO VIAL RUTINARIO CAMINO VECINAL EMP. AR-111 (NUEVO SIBAYO) - TUTI - EMP.AR-681 (DV. CHIVAY), DISTRITOS DE CHIVAY, TUTI Y SIBAYO, PROVINCIA DE CAYLLOMA, DEPARTAMENTO DE AREQUIPA', 'longitud': 32252.0}, 'datos_generales': {'distritos': ['Chivay', 'Tuti', 'Sibayo'], 'region': 'Arequipa', 'provincia': 'Caylloma'}, 'contrato': {'jefe_mantenimiento': {'titulo': 'Ingeniero', 'nombre': 'Genaro', 'dni': 0, 'apellido': 'Tinta Cáceres'},

## Cargando cargas de trabajo del contratista

In [185]:
colec_raiz = "rutinarios"
doc_proyecto = proyecto  # puede ser tu variable proyecto
colec_valoriz = "valorizaciones"
id_valoriz = "4"
doc_ref_valorizacion = (
    db.collection(colec_raiz)
    .document(doc_proyecto)
    .collection(colec_valoriz)
    .document(id_valoriz)
)

In [186]:
valorizacion_snapshot = doc_ref_valorizacion.get()
if not valorizacion_snapshot.exists:
    print("No existe el documento de cargas de trabajo.")

cargas_trabajo_contratista = valorizacion_snapshot.to_dict()['cargas_trabajo_contratista']
print("Cargas de trabajo:", cargas_trabajo_contratista)

Cargas de trabajo: {'MR101': 0.86, 'MR202': 6.0, 'MR301': 3600.0, 'MR601': 64.5, 'MR201': 2720.0, 'MR401': 5}


In [187]:
progresiva_inicio=data_mantenimiento.to_dict()["expediente"]["coordenadas"]["inicio"]["progresiva"]
progresiva_fin=data_mantenimiento.to_dict()["expediente"]["coordenadas"]["fin"]["progresiva"]

print(progresiva_inicio)
print(progresiva_fin)

0
32252.0


## Cargas de trabajo programadas

In [188]:
# Concatenar la ruta completa al archivo .pkl
ruta_archivo = os.path.join("data",proyecto, f"{proyecto}_cargas_trabajo.pkl")

# Leer el archivo pickle
with open(ruta_archivo, "rb") as f:
    cargas_trabajo_programadas_anualmente = pickle.load(f)

cargas_trabajo_programadas_anualmente.head(15)

,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,TOTAL
codigo,,,,,,,,,,
MR101,0.856667,1.713333,0.856667,1.713333,1.713333,0.856667,0.856667,0.856667,0.856667,10.28
MR102,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR103,0.000000,13.332500,13.332500,0.000000,0.000000,13.332500,0.000000,13.332500,0.000000,53.33
MR104,0.000000,0.000000,14.221667,14.221667,14.221667,14.221667,14.221667,14.221667,0.000000,85.33
MR111,92.230488,202.907073,184.460976,202.907073,202.907073,184.460976,202.907073,184.460976,55.338293,1512.58
MR112,52.500000,87.500000,87.500000,87.500000,87.500000,87.500000,87.500000,87.500000,35.000000,700.00
MR201,2205.681818,4411.363636,3676.136364,4411.363636,4411.363636,3676.136364,4411.363636,3676.136364,1470.454545,32350.00
MR202,3.059118,5.098529,4.078824,4.078824,4.078824,4.078824,4.078824,4.078824,2.039412,34.67
MR203,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [189]:
#eliminando el total
cargas_trabajo_programadas_anualmente = cargas_trabajo_programadas_anualmente.iloc[:-1]


In [190]:
cargas_trabajo_programadas_anualmente.index.to_list()

['MR101',
 'MR102',
 'MR103',
 'MR104',
 'MR111',
 'MR112',
 'MR201',
 'MR202',
 'MR203',
 'MR204',
 'MR205',
 'MR206',
 'MR301',
 'MR401',
 'MR501',
 'MR601',
 'MR701',
 'MR702']

## Cargando cronograma anual

In [191]:
# Concatenar la ruta completa al archivo .pkl
ruta_archivo = os.path.join("data",proyecto, f"{proyecto}_cronograma_anual.pkl")

# Leer el archivo pickle
with open(ruta_archivo, "rb") as f:
    df_cronograma_anual = pickle.load(f)

df_cronograma_anual.head(15)

,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,TOTAL
MR101,1,2,1,2,2,1,1,1,1,12
MR102,0,0,0,0,0,0,0,0,0,0
MR103,0,1,1,0,0,1,0,1,0,4
MR104,0,0,1,1,1,1,1,1,0,6
MR111,5,11,10,11,11,10,11,10,3,82
MR112,3,5,5,5,5,5,5,5,2,40
MR201,3,6,5,6,6,5,6,5,2,44
MR202,3,5,4,4,4,4,4,4,2,34
MR203,0,0,0,0,0,0,0,0,0,0
MR204,0,0,0,0,0,0,0,0,0,0


## Cargando actividades

In [192]:
ruta_actividades= os.path.join("data", "general_data", "actividades.json")
with open(ruta_actividades, 'r', encoding='utf-8') as archivo:
    actividades = json.load(archivo)
# Ahora 'datos' es un diccionario de Python
print(actividades)

[{'key': 'MR100', 'value': {'label': 'Conservación de calzada', 'value': [{'key': 'MR101', 'value': {'label': 'Limpieza de calzada', 'carga_trabajo': 0, 'unidad': 'Km'}}, {'key': 'MR102', 'value': {'label': 'Bacheo', 'carga_trabajo': 0, 'unidad': 'm2'}}, {'key': 'MR103', 'value': {'label': 'Desquinche', 'carga_trabajo': 0, 'unidad': 'm3'}}, {'key': 'MR104', 'value': {'label': 'Remoción de derrumbes', 'carga_trabajo': 0, 'unidad': 'm3'}}]}}, {'key': 'MR200', 'value': {'label': 'Limpieza de obras de arte', 'value': [{'key': 'MR201', 'value': {'label': 'Limpieza de cunetas', 'carga_trabajo': 1440, 'unidad': 'm'}}, {'key': 'MR202', 'value': {'label': 'Limpieza de alcantarillas', 'carga_trabajo': 0, 'unidad': 'unidad'}}, {'key': 'MR203', 'value': {'label': 'Limpieza de badén', 'carga_trabajo': 0, 'unidad': 'm2'}}, {'key': 'MR204', 'value': {'label': 'Limpieza de zanjas de coronación', 'carga_trabajo': 0, 'unidad': 'm'}}, {'key': 'MR205', 'value': {'label': 'Limpieza de pontones', 'carga_tra

## Funciones necesarias

In [193]:
def ordenar_por_centena(data: Any) -> List[Dict[str, Any]]:
    """
    Toma un dict (o un JSON en formato str) cuyas claves acaban en número
    y devuelve una lista de dicts {'key':…, 'value':…} ordenada por ese número.
    Compatible con Firestore (to_dict()) y JSON.
    """
    # Si viene como cadena JSON, lo convertimos
    if isinstance(data, str):
        data = json.loads(data)
    
    # Aseguramos que sea dict
    if not isinstance(data, dict):
        raise ValueError("Se esperaba un diccionario o un string JSON que represente un diccionario.")

    pattern = re.compile(r'(\d+)$')

    def obtener_clave_numerica(item):
        clave = str(item[0])  # Convertimos clave a string en caso no lo sea
        match = pattern.search(clave)
        if match:
            return int(match.group(1))
        else:
            return float('inf')  # Opcional: claves sin número al final se van al final

    sorted_items = sorted(data.items(), key=obtener_clave_numerica)

    return [{"key": k, "value": v} for k, v in sorted_items]

In [194]:
def clave_grupo(item):
    return int(item['key'][2:]) // 100  # 101→1, 201→2, etc.

In [195]:
def calcular_avance(programado, ejecutado):
    """
    Calcula el porcentaje de avance basado en lo programado y lo ejecutado.
    
    Si lo programado es 0 y lo ejecutado es mayor a 0, devuelve 'ejecución adelantada'.
    Si ambos son 0, devuelve 0.0.
    En cualquier otro caso, devuelve el porcentaje (0-100) como float redondeado a 2 decimales.
    """
    if programado == 0:
        if ejecutado > 0:
            return "Ejec. adelantada"
        else:
            return 0.0
    else:
        porcentaje = (ejecutado / programado) 
        return round(porcentaje, 2)


In [196]:
def formatear_progresiva(distancia, decimales=0):
    """
    Convierte una distancia en metros a notación de progresiva.
    
    Parámetros:
    - distancia: int o float, la distancia en metros.
    - decimales: int, número de decimales a mostrar en la parte de los metros.
    
    Retorna:
    - str: progresiva en formato 'K+XXX' con los decimales indicados.
    """
    if not isinstance(distancia, (int, float)):
        raise ValueError("La distancia debe ser un número (int o float).")
    if not isinstance(decimales, int) or decimales < 0:
        raise ValueError("Los decimales deben ser un entero no negativo.")
    
    km = int(distancia) // 1000
    metros = distancia - (km * 1000)
    
    formato_metros = f"{metros:0.{decimales}f}".zfill(3 + (1 if decimales > 0 else 0) + decimales)
    return f"{km}+{formato_metros}"

In [197]:
cargas_trabajo_ordenadas=ordenar_por_centena(cargas_trabajo_contratista)
print(cargas_trabajo_ordenadas)

[{'key': 'MR101', 'value': 0.86}, {'key': 'MR201', 'value': 2720.0}, {'key': 'MR202', 'value': 6.0}, {'key': 'MR301', 'value': 3600.0}, {'key': 'MR401', 'value': 5}, {'key': 'MR601', 'value': 64.5}]


In [198]:
type(cargas_trabajo_ordenadas[0])

dict

## Escribiendo el excel

### Path

In [199]:
# Ruta dinámica
ruta_directorio = os.path.join("output", proyecto, str(mes))
ruta_archivo = os.path.join(ruta_directorio, "cargas_trabajo.xlsx")
# Crear carpetas si no existen
os.makedirs(ruta_directorio, exist_ok=True)

wb = xlsxwriter.Workbook(ruta_archivo)
ws=wb.add_worksheet("cargas_trabajo")

### Formatos

In [200]:
# 1. Propiedades base
BASE_CELL = {
    "valign": "vcenter",
    "border": 1,
}

BASE_HEADER = {
    **BASE_CELL,
    "bold": True,
}

BG_HEADER = "#D9E1F2"
BG_TABLE = "#C5D9F1"


# 2. Función de helper para crear formatos
def fmt(wb, **props):
    cfg = {}
    # parte genérica
    cfg.update(props)
    return wb.add_format(cfg)


# 3. Diccionario de formatos
formats = {
    # encabezados
    "header": {"align": "center", "bg_color": BG_HEADER, **BASE_HEADER},
    "header2": {"align": "left", **BASE_HEADER},
    "header_label": {"align": "center", **BASE_HEADER},
    # formato vertical header
    "header_vertical": {
        **BASE_HEADER,
        "bold": True,
        "align": "center",
        "valign": "vcenter",
        "bg_color": BG_HEADER,
        "rotation": 90,  # <— ¡aquí está el giro!
    },
    # encabezados de tabla
    "table_header": {
        "align": "center",
        "text_wrap": True,
        "shrink": True,
        "bg_color": BG_TABLE,
        **BASE_CELL,
    },
    "table_header_int": {"align": "center", "bg_color": BG_TABLE, **BASE_CELL},
    # celdas numéricas y de texto
    "cell": {"font_size": 8, "num_format": "#,##0.00", **BASE_CELL},
    # "cell": {"font_size": 8, "num_format": "#,##0.00", **BASE_CELL},
    "cell_int": {"num_format": "#,##0", **BASE_CELL},
    "cell_number": {"num_format": "#,##0.00", **BASE_CELL},
    "cell_number_percent": {
        "num_format": "#,##0.00%",
        "align": "right",
        **BASE_CELL,
    },
    "cell_text": {"align": "left", "text_wrap": True, "shrink": True, **BASE_CELL},
    "cell_right": {"align": "right", **BASE_CELL},
    # sumas
    "suma": {"num_format": "#,##0.00", **BASE_CELL},
    # soles
    "soles_color": {
        "bold": True,
        "bg_color": BG_HEADER,
        "num_format": '"S/." #,##0.00',
        **BASE_CELL,
    },
    "soles": {"num_format": '"S/." #,##0.00', **BASE_CELL},
    # formato de ajuste general
    "ajustar": {"align": "center", "text_wrap": True, "shrink": True, **BASE_CELL},
    "only_border": {
        "border": 1,
    },
}

# 4. Creación dinámica de los objetos Format
wb_formats = {name: fmt(wb, **props) for name, props in formats.items()}

# Ahora sólo usa wb_formats["header"], wb_formats["cell_text"], etc.

In [201]:
# Punto de inserción (cero-indexed)
table_start_row = 6   # si quieres que empiece en la fila 7
table_start_col = 1   # si quieres que empiece en la columna B

In [202]:
def merge(ws, r1, c1, r2, c2, valor, fmt):
    """Merge relative a table_start_* + offsets."""
    ws.merge_range(
        xl_range(table_start_row + r1,
                 table_start_col + c1,
                 table_start_row + r2,
                 table_start_col + c2),
        valor, fmt
    )

In [203]:
def write_rel(ws, row_offset, col_offset, value, fmt=None):
    """
    Escribe en la celda (table_start_row + row_offset, table_start_col + col_offset)
    """
    abs_row = table_start_row + row_offset
    abs_col = table_start_col + col_offset
    ws.write(abs_row, abs_col, value, fmt)


In [204]:
# 2. Helpers con tuplas (x, y)

def merge_xy(ws, start: tuple[int,int], end: tuple[int,int], value, fmt):
    """
    Merge de celdas desde start=(x1, y1) hasta end=(x2, y2),
    relativas a table_start_col/row.
    """
    (x1, y1), (x2, y2) = start, end
    ws.merge_range(
        xl_range(
            table_start_row + y1,  # row1 absoluto
            table_start_col + x1,  # col1 absoluto
            table_start_row + y2,  # row2 absoluto
            table_start_col + x2,  # col2 absoluto
        ),
        value, fmt
    )

def write_xy(ws, pos: tuple[int,int], value, fmt=None):
    """
    Escribe un valor en pos=(x, y) relativa a table_start_col/row.
    """
    x, y = pos
    ws.write(
        table_start_row + y,
        table_start_col + x,
        value, fmt
    )

In [205]:
def set_columns_rel(ws, rel_col_widths):
    """
    Aplica ws.set_column a rangos de columnas definidos
    en coordenadas relativas (x) a table_start_col.
    rel_col_widths: lista de tuplas (first_rel, last_rel, width)
    """
    for first_rel, last_rel, width in rel_col_widths:
        abs_first = table_start_col + first_rel
        abs_last  = table_start_col + last_rel
        ws.set_column(abs_first, abs_last, width)

In [206]:
col_widths_rel = [
    (0, 0, 8),  # columna B en adelante:  B–B ancho=8
    (1, 1, 12),  # C–C ancho=30
    (2, 2, 30),  # D–H ancho=8
    (3, 3, 12),  # I–T ancho=8
    (4, 4, 10),  # U–U ancho=10
    (5, 5, 10),  # U–U ancho=10
    (6, 6, 14),  # U–U ancho=10
    (7, 7, 14),
]

set_columns_rel(ws, col_widths_rel)

In [207]:
merge_xy(ws, (0, 0), (0, 1), "N.º", wb_formats["header"])
merge_xy(ws, (1, 0), (1, 1), "Código", wb_formats["header"])
merge_xy(ws, (2, 0), (2, 1), "Actividades", wb_formats["header"])

merge_xy(ws, (3, 0), (3, 1), "Unidad", wb_formats["header"])

merge_xy(ws, (4, 0), (5, 0), "Cargas de trabajo", wb_formats["header"])

write_xy(ws, (4, 1), "Programación", wb_formats["header_vertical"])
write_xy(ws, (5, 1), "Ejecución", wb_formats["header_vertical"])

merge_xy(ws, (6, 0), (6, 1), "Avance", wb_formats["header"])
merge_xy(ws, (7, 0), (7, 1), "Progresivas", wb_formats["header"])


In [208]:
current_col = 0
current_row = 2

In [209]:
print(current_row, current_col)

2 0


In [210]:
codigo = "MR102"
mes_int = 4
anio = 2025

# Formatea el mes con dos dígitos y crea "2025-04"
col = f"{anio}-{mes_int:02d}"

valor = cargas_trabajo_programadas_anualmente.loc[codigo, col]
print(type(valor))  # 153.14833…

<class 'numpy.float64'>


In [211]:
columnas_insertadas = 0

impresos = set()

contador_items = 1

for grupo, elementos in itertools.groupby(cargas_trabajo_ordenadas, clave_grupo):
    current_col = 0

    # codigo=int(carga_trabajo['key'][2:])
    # grupo=codigo//100
    # print(grupo)

    inicio = grupo * 100
    fin = inicio + 99

    print(f"Valores entre {inicio} al {fin}!")
    resultado = next(
        (item for item in actividades if item["key"] == f"MR{inicio}"), None
    )
    # print(resultado)

    write_xy(ws, (current_col, current_row), "", wb_formats["cell_text"])  # numero
    current_col += 1
    write_xy(
        ws, (current_col, current_row), resultado["key"], wb_formats["cell_text"]
    )  # codigo
    current_col += 1
    write_xy(
        ws,
        (current_col, current_row),
        resultado["value"]["label"],
        wb_formats["cell_text"],
    )  #

    current_row += 1

    for it in elementos:
        current_col = 0
        # print(f"  {it['key']}: {it['value']}")
        print(f"  {it['key']}: {it['value']}")

        resultado2 = next(
            (item for item in resultado["value"]["value"] if item["key"] == it["key"]),
            None,
        )
        print(resultado)

        write_xy(
            ws, (current_col, current_row), contador_items, wb_formats["cell_int"]
        )  # numero
        current_col += 1
        write_xy(
            ws, (current_col, current_row), resultado2["key"], wb_formats["cell_text"]
        )
        current_col += 1
        write_xy(
            ws,
            (current_col, current_row),
            resultado2["value"]["label"],
            wb_formats["cell_text"],
        )
        current_col += 1
        write_xy(
            ws,
            (current_col, current_row),
            resultado2["value"]["unidad"],
            wb_formats["cell_text"],
        )
        current_col += 1
        write_xy(
            ws,
            (current_col, current_row),
            cargas_trabajo_programadas_anualmente.loc[it["key"], f"{anio}-{mes:02d}"],
            wb_formats["cell_number"],
        )
        current_col += 1
        write_xy(
            ws,
            (current_col, current_row),
            cargas_trabajo_contratista[it["key"]],
            wb_formats["cell_number"],
        )
        current_col += 1

        avance = calcular_avance(
            programado=cargas_trabajo_programadas_anualmente.loc[it["key"], f"{anio}-{mes:02d}"],
            ejecutado=cargas_trabajo_contratista[it["key"]],
        )
        write_xy(
            ws,
            (current_col, current_row),
            avance,
            wb_formats["cell_number_percent"],
        )
        current_col += 1

        progresivas = (
            formatear_progresiva(progresiva_inicio)
            + " - "
            + formatear_progresiva(progresiva_fin)
        )
        write_xy(
            ws,
            (current_col, current_row),
            progresivas,
            wb_formats["cell_number_percent"],
        )
        current_col += 1

        #######3
        contador_items += 1
        current_row += 1

Valores entre 100 al 199!
  MR101: 0.86
{'key': 'MR100', 'value': {'label': 'Conservación de calzada', 'value': [{'key': 'MR101', 'value': {'label': 'Limpieza de calzada', 'carga_trabajo': 0, 'unidad': 'Km'}}, {'key': 'MR102', 'value': {'label': 'Bacheo', 'carga_trabajo': 0, 'unidad': 'm2'}}, {'key': 'MR103', 'value': {'label': 'Desquinche', 'carga_trabajo': 0, 'unidad': 'm3'}}, {'key': 'MR104', 'value': {'label': 'Remoción de derrumbes', 'carga_trabajo': 0, 'unidad': 'm3'}}]}}
Valores entre 200 al 299!
  MR201: 2720.0
{'key': 'MR200', 'value': {'label': 'Limpieza de obras de arte', 'value': [{'key': 'MR201', 'value': {'label': 'Limpieza de cunetas', 'carga_trabajo': 1440, 'unidad': 'm'}}, {'key': 'MR202', 'value': {'label': 'Limpieza de alcantarillas', 'carga_trabajo': 0, 'unidad': 'unidad'}}, {'key': 'MR203', 'value': {'label': 'Limpieza de badén', 'carga_trabajo': 0, 'unidad': 'm2'}}, {'key': 'MR204', 'value': {'label': 'Limpieza de zanjas de coronación', 'carga_trabajo': 0, 'unidad

In [212]:
print(current_row, current_col)

13 8


In [213]:
print(
    xl_range(
        table_start_row,
        table_start_col,
        table_start_row + current_row-1,
        table_start_col+current_col-1,
    ),
)

B7:I19


In [214]:
# colocar grilla  a toda la tabla
ws.conditional_format(
    xl_range(
        table_start_row,
        table_start_col,
        table_start_row + current_row - 1,
        table_start_col + current_col - 1,
    ),
    {"type": "no_errors", "format": wb_formats["only_border"]},
)

0

In [215]:
# Guardar archivo
wb.close()

## Imprimiendo una imagen a partir del excel

In [216]:
# 1. Carga el libro y la hoja

wb = load_workbook(ruta_archivo, data_only=True)
ws = wb.active  # o wb["NombreHoja"]

# 2. Determina el rango usado
min_row, min_col = ws.min_row, ws.min_column
max_row, max_col = ws.max_row, ws.max_column
rango = f"{get_column_letter(min_col)}{min_row}:{get_column_letter(max_col)}{max_row}"

# 3. Exporta a PNG
salida_png = f"cargas_trabajo.png"
ruta_salida_image= os.path.join(ruta_directorio, salida_png)
excel2img.export_img(ruta_archivo, ruta_salida_image, ws.title, rango)

print(f"Imagen generada: {salida_png}")


Imagen generada: cargas_trabajo.png
